In [1]:
using DataFrames, MLDataUtils
using Clustering, Distances
using CSV
using Random
using Logging

# Set up Logging - we recommend to use this command to avoid package warnings during the model training process.
logger = Logging.SimpleLogger(stderr, Logging.Warn);
global_logger(logger);

In [2]:
#### Set parameters for the learners
cr = :silhouette
method = "ICOT_local"
warm_start = :oct; # ???????
geom_search = true ## ?????
threshold = 0.99
seed = 1
gridsearch = false
num_tree_restarts = 10
complexity_c = 0.0
min_bucket = 100
maxdepth = 3

###### Step 1: Prepare the data
# Read the data - recommend the use of the (deprecated) readtable() command to avoid potential version conflicts with the CSV package.
# data = readtable("../data/ruspini.csv"); 


In [3]:
data = DataFrame(CSV.File("/home/sfy/Documents/VScodeProject/Thesis/algorithms/alldata.csv"))

print(size(data))

# Convert the dataset to a matrix
data_array = Matrix(data)
# Get the number of observations and features
n, p = size(data_array)
data_t = data_array';

##### Step 2: Fit K-means clustering on the dataset to generate a warm-start for ICOT
#Fix the seed
Random.seed!(seed);
K = 2

# Run k-means and save the assignments 
kmeans_result = kmeans(data_t, K);
assignment = kmeans_result.assignments;


data_full = DataFrame(hcat(data, assignment, makeunique=true));
# names!(data_full, [:x1, :x2, :true_labels, :kmean_assign]);
# X = data_full[:, 1:2];
# y = data_full[:, :true_labels];

# Prepare data for ICOT: features are stored in the matrix X, and the warm-start labels are stored in y

# get x and y
X = select(data, Not([:y]))
y = data[:, :y]

(41257, 697)

41257-element Array{Int64,1}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 1
 1
 1
 1
 1
 1
 1
 1
 1

In [4]:
##### Step 3a. Before running ICOT, start by testing the IAI license
lnr_oct = ICOT.IAI.OptimalTreeClassifier(localsearch=false, max_depth=maxdepth,
    minbucket=min_bucket,
    criterion=:misclassification
)
grid = ICOT.IAI.GridSearch(lnr_oct)
ICOT.IAI.fit!(grid, X, y)

┌ Warning: This copy of Interpretable AI software is for academic purposes only and not for commercial use.
└ @ IAIBase /home/iai/.julia/packages/IAIBase/pOrUV/src/precompile.jl:19


All Grid Results:

│ Row │ cp         │ train_score │ valid_score │ rank_valid_score │
│     │ Float64    │ Float64     │ Float64     │ Int64            │
├─────┼────────────┼─────────────┼─────────────┼──────────────────┤
│ 1   │ 0.00219543 │ 0.859245    │ 0.86079     │ 1                │

Best Params:
  cp => 0.0021954279342068506

Best Model - Fitted OptimalTreeClassifier:
  1) Split: android.permission.READ_PHONE_STATE < 0.5
    2) Split: android.intent.category.HOME < 0.5
      3) Split: android.permission.SEND_SMS < 0.5
        4) Predict: 0 (96.48%), [11692,427], 12119 points, error 427
        5) Predict: 1 (68.60%), [130,284], 414 points, error 130
      6) Split: android/content/pm/PackageManager;getPackageInfo < 1.5
        7) Predict: 1 (96.07%), [20,489], 509 points, error 20
        8) Predict: 0 (88.70%), [102,13], 115 points, error 13
    9) Split: android/util/Base64;decode < 3.5
      10) Split: android/media/MediaRecorder;setAudioSource < 0.5
        11) Predict: 1 (

In [6]:
ICOT.IAI.showinbrowser(grid.lnr)

┌ Warning: `showinbrowser(args...; kwargs...)` is deprecated, use `show_in_browser(args...; kwargs...)` instead.
│   caller = top-level scope at none:0
└ @ Core none:0


"/tmp/tmpj9cMQP/tree.html"

In [24]:

##### Step 3b. Run ICOT

# Run ICOT with no warm-start: 
# warm_start = :none
# lnr_ws_none = ICOT.InterpretableCluster(ls_num_tree_restarts=num_tree_restarts, ls_random_seed=seed, cp=complexity_c, max_depth=maxdepth,
#     minbucket=min_bucket, criterion=cr, ls_warmstart_criterion=cr, kmeans_warmstart=warm_start,
#     geom_search=geom_search, geom_threshold=threshold);
# run_time_icot_ls_none = @elapsed ICOT.fit!(lnr_ws_none, X, y);

# ICOT.showinbrowser(lnr_ws_none)
# 30 mins without results

# score_ws_none = ICOT.score(lnr_ws_none, X, y, criterion=:dunnindex);
# score_al_ws_none = ICOT.score(lnr_ws_none, X, y, criterion=:silhouette);


In [5]:

# Run ICOT with an OCT warm-start: fit an OCT as a supervised learning problem with labels "y" and use this as the warm-start
warm_start = :oct

min_bucket = 5000 #now is 11 # require ** leaves in each cluster
# alpha: 1: for numerical covariates 0 for categorical !0 


lnr_ws_oct = ICOT.InterpretableCluster(ls_num_tree_restarts=num_tree_restarts, ls_random_seed=seed, cp=complexity_c, max_depth=maxdepth,
    minbucket=min_bucket, criterion=cr, ls_warmstart_criterion=cr, kmeans_warmstart=warm_start,
    geom_search=geom_search, geom_threshold=threshold);
run_time_icot_ls_oct = @elapsed ICOT.fit!(lnr_ws_oct, X, y);



In [ ]:

score_ws_oct = ICOT.score(lnr_ws_oct, X, y, criterion=:dunnindex);
score_al_ws_oct = ICOT.score(lnr_ws_oct, X, y, criterion=:silhouette);